In [1]:
import os
import uproot
from ROOT import TFile
import awkward as ak
from config.utils import lVector

Welcome to JupyROOT 6.30/04


In [2]:
def runOneFile(inputfile, outputfilesDir):
   isdata=False
   if "Data"  in inputfile.split("/")[-1]:
      isdata=True
   print ("Status of the isdata flag:", isdata)
   outputfile = outputfilesDir+"/"+inputfile.split("/")[-1]
   mycache = uproot.LRUArrayCache("500 MB")

   file_=uproot.open(inputfile)

   print ("root file opened: ", inputfile)
   print(file_.keys())

   fulltree_=ak.ArrayBuilder()
   niterations=0
   for tree_ in uproot.iterate(file_["DiphotonTree/data_125_13TeV_NOTAG"], ["run", "lumi", "event", "lead_pt", "lead_eta", "lead_phi","sublead_pt", "sublead_eta", "sublead_phi","weight","weight_central",], step_size=50000):
      print ("tree length for iteratiion ", len(tree_), (niterations))
      niterations=niterations+1
      cms_events = ak.zip({"run":tree_["run"],"lumi":tree_["lumi"],"event": tree_["event"],"lead_pt":tree_["lead_pt"], "lead_eta":tree_["lead_eta"], "lead_phi":tree_["lead_phi"], "sublead_pt":tree_["sublead_pt"], "sublead_eta":tree_["sublead_eta"], "sublead_phi":tree_["sublead_phi"],"weight_central":tree_["weight_central"],"weight":tree_["weight"]},depth_limit=1)
      out_events = ak.zip({"run":tree_["run"],"lumi":tree_["lumi"],"event": tree_["event"]},depth_limit=1)

      diJet_mass = lVector(cms_events["lead_pt"], cms_events["lead_eta"], cms_events["lead_phi"],cms_events["sublead_pt"],cms_events["sublead_eta"],cms_events["sublead_phi"])
      cms_events["dijet_mass"] = diJet_mass

      from regions import get_mask_preselection
      cms_events["mask_preselection"]   = get_mask_preselection(cms_events)

      out_events["dijet_mass"] = cms_events["dijet_mass"]
      out_events["preselection"] = cms_events["mask_preselection"]
      print('hello')
      fulltree_=ak.concatenate([out_events,fulltree_],axis=0)

   # from variables import vardict, regions
   # from binning import binning
   # print ("NOW ENTERING THE HISTOGRAMMING")
   # f = TFile(outputfile,"RECREATE")
   # for ireg in regions:
   #    thisregion  = fulltree_[fulltree_[ireg]==True]
   #    thisregion_ = thisregion[~(ak.is_none(thisregion))]
   #    weight_ = "weight_"+ireg
   #    for ivar in variables_common[ireg]:
   #       hist_name_ = "h_reg_"+ireg+"_"+vardict[ivar]
   #       h = VarToHist(thisregion_[ivar], thisregion_[weight_], hist_name_, binning[ireg][ivar])
   #       f.cd()
   #       h.Write()

In [3]:
inputfilesDir = '/Users/ptiwari/cmscern/eos/DoNotSync/hhtobbgg/HiggsDNA_root/v1/Run3_2022postEE_merged'
output_dir = "outputfiles"
if not os.path.exists(output_dir):
   os.makedirs(output_dir)

inputfiles = [f"{inputfilesDir}/{infile_}" for infile_ in os.listdir(inputfilesDir) if infile_.endswith('.root')]

def main():
   # for infile_ in inputfiles:
   #    runOneFile(infile_, output_dir)
   runOneFile(inputfiles[0], output_dir)
if __name__ == "__main__":
   main()

Status of the isdata flag: True
root file opened:  /Users/ptiwari/cmscern/eos/DoNotSync/hhtobbgg/HiggsDNA_root/v1/Run3_2022postEE_merged/Data_EraE.root
['DiphotonTree;1', 'DiphotonTree/data_125_13TeV_NOTAG;1']
tree length for iteratiion  47355 0


TypeError: ones_like() got an unexpected keyword argument 'shape'

This error occurred while calling

    ak.ones_like(
        <Array [167, 93.3, 99.5, ..., 115, 103, 133] type='47355 * float64'>
        shape = 47355
        dtype = type-instance
    )